<a href="https://colab.research.google.com/github/jisoolee11/AI_study/blob/main/RESNET34_CIFAR10(test2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 불러오기
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# CIFAR10 불러오고 정규화하기
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary



# class VGG16(nn.Module):
#     def __init__(
#         self,
#         in_channels: int = 3) -> None:
#         super().__init__()
#         self.conv1_1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
#         self.conv1_2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)

#         self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv2_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

#         self.conv3_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.conv3_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
#         self.conv3_3 = nn.Conv2d(128, 128, kernel_size=1, padding=1)

#         self.conv4_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
#         self.conv4_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
#         self.conv4_3 = nn.Conv2d(256, 256, kernel_size=1, padding=1)

#         self.conv5_1 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
#         self.conv5_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
#         self.conv5_3 = nn.Conv2d(256, 256, kernel_size=1, padding=1)

#         self.pool = nn.AdaptiveAvgPool2d((2, 2))

#         self.fc6 = nn.Linear(in_features=256*2*2, out_features=512)
#         self.fc7 = nn.Linear(in_features=512, out_features=512)
#         self.fc8 = nn.Linear(in_features=512, out_features=10)
        
    
#     def forward(self, x):
#       x = F.relu(self.conv1_1(x))
#       x = F.relu(self.conv1_2(x))
#       x = self.pool(x)
#       x = F.relu(self.conv2_1(x))
#       x = F.relu(self.conv2_2(x))
#       x = self.pool(x)
#       x = F.relu(self.conv3_1(x))
#       x = F.relu(self.conv3_2(x))
#       x = F.relu(self.conv3_3(x))
#       x = self.pool(x)
#       x = F.relu(self.conv4_1(x))
#       x = F.relu(self.conv4_2(x))
#       x = F.relu(self.conv4_3(x))
#       x = self.pool(x)
#       x = F.relu(self.conv5_1(x))
#       x = F.relu(self.conv5_2(x))
#       x = F.relu(self.conv5_3(x))
#       x = self.pool(x)

#       x = x.view(-1, 2*2*256)

#       x = F.relu(self.fc6(x))
#       x = F.relu(self.fc7(x))
#       x = F.relu(self.fc8(x))
      
#       return x




In [4]:
from torchvision.models import resnet34

model = resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [5]:
import torch.optim as optim

criterion = nn. CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [6]:
model.train()
for epoch in range(2):
    # running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, loss.item()))

print('Finished Training')

[1,     1] loss: 2.711
[1,   101] loss: 1.155
[1,   201] loss: 0.963
[1,   301] loss: 0.857
[1,   401] loss: 0.783
[2,     1] loss: 0.649
[2,   101] loss: 0.604
[2,   201] loss: 0.499
[2,   301] loss: 0.594
[2,   401] loss: 0.562
Finished Training


In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 81 %
Accuracy of   car : 90 %
Accuracy of  bird : 63 %
Accuracy of   cat : 67 %
Accuracy of  deer : 71 %
Accuracy of   dog : 59 %
Accuracy of  frog : 86 %
Accuracy of horse : 84 %
Accuracy of  ship : 87 %
Accuracy of truck : 72 %
